In [4]:
import os

ENV = os.environ.get('ENV')
BUCKET = None
HOST = None
print(ENV)

if ENV == None:

    HOST = 'minio'
    BUCKET = 'data-product-a-dev'

if ENV == 'PRD':

    HOST = 'minio'
    BUCKET = 'data-product-a'

print(BUCKET)
print(HOST)

None
data-product-a-dev
minio


In [14]:
import time
from pyspark.sql import SparkSession





spark = SparkSession.builder.config('spark.jars', """./home/jars/aws-java-sdk-bundle-1.12.419.jar,
                                                     ./home/jars/hadoop-aws-3.3.2.jar,
                                                     ./home/jovyan/jars/aws-java-sdk-bundle-1.12.419.jar,
                                                     ./home/jovyan/jars/hadoop-aws-3.3.2.jar""") \
                        .getOrCreate()

MINIO_ACCESSKEY = "TclV2UThQ98tgWko"
MINIO_SECRETKEY = "jOi3VpAUZZYn6tdkUUMqj5DfQFYt9BnU"


spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", f"http://{HOST}:9000")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", MINIO_ACCESSKEY)
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", MINIO_SECRETKEY)
spark._jsc.hadoopConfiguration().set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "true")
spark._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
spark._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
spark._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
spark._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")

In [16]:
print(spark.version)

3.3.2


In [17]:
from pyspark.sql import functions as f

In [18]:
df_bronze = spark.read.csv(f's3a://{BUCKET}/bronze/', sep=';', header=True)
df_bronze.show()

+--------+--------------------+-------+-----------+------------+--------+---------+-----------------+----------------+-------------+-----+-----------------------+-----------------------+--------+
|IBGECode|         LocalCidade|LocalUF|LocalEstado|RegiaoBrasil|Latitude|Longitude|        Gentilico|PopEstimada_2018|PopCenso 2010| IDHM|ReceitasRealizadas_2014|DespesasEmpenhadas_2014|Pib_2014|
+--------+--------------------+-------+-----------+------------+--------+---------+-----------------+----------------+-------------+-----+-----------------------+-----------------------+--------+
| 1100015|ALTA FLORESTA D'O...|     RO|   Rondonia|           N|-119.283| -619.953|   alta-florense |           23167|        24392|0,641|                  53490|                  47577|16461,11|
| 1100023|           ARIQUEMES|     RO|   Rondonia|           N|-990.571| -630.325|     ariquemense |          106168|        90353|0,702|                      0|                      0|19401,43|
| 1100031|          

In [19]:
df_bronze.write.parquet(f's3a://{BUCKET}/silver/brazilian_cities1/', mode='overwrite')

In [12]:
df_bronze.groupBy('LocalUF').agg((f.count('*')).alias('qtd')).write.parquet(f's3a://{BUCKET}/gold/state_count3', mode='overwrite')

In [13]:
spark.read.parquet(f's3a://{BUCKET}/gold/state_count').show()

+-------+---+
|LocalUF|qtd|
+-------+---+
|     SC|295|
|     RO| 52|
|     PI|224|
|     AM| 62|
|     RR| 15|
|     GO|246|
|     TO|139|
|     MT|141|
|     SP|645|
|     PB|223|
|     ES| 78|
|     RS|497|
|     MS| 79|
|     AL|102|
|     MG|853|
|     PA|144|
|     BA|417|
|     SE| 75|
|     PE|185|
|     CE|184|
+-------+---+
only showing top 20 rows

